In [1]:
import spacy

#import model as model_config
#from data_utils import load as load_data, extract_features
import numpy as np

%load_ext autoreload
%autoreload 2




import spacy
import pickle
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import LabelEncoder


twitter_data_neg_small = './twitter-datasets/train_neg.txt'
twitter_data_pos_small = './twitter-datasets/train_pos.txt'
twitter_data_small     = './twitter-datasets/train_small.txt'

twitter_data_neg_full  = './twitter-datasets/train_neg_full.txt'
twitter_data_pos_full  = './twitter-datasets/train_pos_full.txt'
twitter_data_full      = './twitter-datasets/train_full.txt'
nlp = spacy.load('en_core_web_lg')

In [15]:
#nlp = spacy.load('en_core_web_lg')

In [33]:
def load_twitter_data_small(from_cache=False):
    cached_data_path = twitter_data_small + '.cached.pkl'

    if from_cache:
        print('Loading data from cache...')
        with open(cached_data_path, 'rb') as f:
            return pickle.load(f)

    max_length = 1000

    print('Loading and preparing data...')
    raw_data_neg = pd.read_csv(twitter_data_neg_small, header=None, sep="\n", encoding='latin1', names=['text'],
                               error_bad_lines=False, warn_bad_lines=False).drop_duplicates()
    raw_data_neg['label'] = 0
#     print(raw_data_neg)
    raw_data_pos = pd.read_csv(twitter_data_pos_small, header=None, sep="\n", encoding='latin1', names=['text'],
                               error_bad_lines=False, warn_bad_lines=False).drop_duplicates()
    raw_data_pos['label'] = 1
    
   # raw_data = raw_data_neg.join(raw_data_pos, on=['text', 'label'])
    
    raw_data = pd.concat([raw_data_neg, raw_data_pos], ignore_index=True)
    
    #print(raw_data)

    # Parse tweet texts
    docs = list(nlp.pipe(raw_data['text'], batch_size=5000, n_threads=2))
    
    print(docs)

#     # Encode labels
#     label_encoder = LabelEncoder()
#     label_encoder.fit(raw_data['gender'])
#     y = label_encoder.transform(raw_data['gender'])

#     # Pull the raw_data into vectors
#     X = extract_features(docs, max_length=max_length)

#     # Split into train and test sets
#     rs = ShuffleSplit(n_splits=2, random_state=42, test_size=0.2)
#     train_indices, test_indices = next(rs.split(X))

#     X_train = X[train_indices]
#     y_train = y[train_indices]
#     X_test = X[test_indices]
#     y_test = y[test_indices]

#     docs = np.array(docs, dtype=object)
#     docs_train = docs[train_indices]
#     docs_test = docs[test_indices]

#     numeric_data = X_train, y_train, X_test, y_test
#     raw_data = docs_train, docs_test, label_encoder

#     with open(cached_data_path, 'wb') as f:
#         pickle.dump((numeric_data, raw_data), f)

#     return numeric_data, raw_data
    return None, None

# Parse tweet texts
#     docs = list(nlp.pipe(raw_data['text'], batch_size=5000, n_threads=8))

#     y = raw_data['polarity'].values

#     # Pull the raw_data into vectors
#     X = extract_features(docs, max_length=max_length)

#     # Split into train and test sets
#     rs = ShuffleSplit(n_splits=2, random_state=42, test_size=0.2)
#     train_indices, test_indices = next(rs.split(X))

#     X_train = X[train_indices]
#     y_train = y[train_indices]
#     X_test = X[test_indices]
#     y_test = y[test_indices]

#     docs = np.array(docs, dtype=object)
#     docs_train = docs[train_indices]
#     docs_test = docs[test_indices]

#     numeric_data = X_train, y_train, X_test, y_test
#     raw_data = docs_train, docs_test, None

#     return numeric_data, raw_data

def load_twitter_data_full(from_cache=False):
    return None, None

def load(data_name, *args, **kwargs):
    load_fn_map = {
        'twitter_data_small': load_twitter_data_small,
        'twitter_data_full': load_twitter_data_full,
    }
    return load_fn_map[data_name](*args, **kwargs)

In [34]:
# Load Twitter data
(_, _, X_test, y_test), (docs_train, docs_test, _) = load('twitter_data_small', from_cache=False)

Loading and preparing data...


KeyboardInterrupt: 